# Программирование на Python
## Семинар 8. Python для анализа данных
#### Задача 1
Вам дан массив данных, представленный в виде списка списков. Каждый вложенный список представляет собой измерение одного параметра для разных индивидуумов. В процессе обработки данных вам часто придется сталкиваться с тем, что параметры могут иметь разную шкалу. Существуют различные способы решения данной проблемы, среди которых - т. н. стандартизация, приводящее значения к стандартному нормальному распределению (большинство значений от -3 до 3). Формула выглядит следующим образом (в данном случае `x` - каждое значение, т. е. операцию ниже необходимо выполнить с каждым значением наших данных):

![standardization](https://i.ibb.co/THbWKqM/1-YSAAU-v-I8-Ol-HQz-G5-A1-Sg.png)

Напишите функцию, которая выполняет эту стандартизацию для двухмерного массива. При этом должна быть предусмотрена возможность осуществить это по строкам, по колонкам и для всего массива в целом.

Решите задачу в двух вариантах: 1) используя ТОЛЬКО встроенные средства Python и 2) с помощью `numpy`. Сравните эффективность двух подходов.

In [1]:
import numpy as np

# (псевдо)случайно генерируем матрицу 10 * 10000 из равномерного распределения с границами от -1000 до 1000
simulated_data = np.random.uniform(-1000, 1000, (10, 10000))
simulated_data_list = simulated_data.tolist()

**Python**

In [3]:
def mean(vector):
    return sum(vector) / len(vector)

def std(vector):
    vector_mean = mean(vector)
    vector_std = (sum([(x - vector_mean) ** 2 for x in vector]) / len(vector)) ** (1 / 2)

    return vector_std

def standardize1d(vector):
    vector_mean = mean(vector)
    vector_sd = std(vector)
    
    vector_std = [(x - vector_mean) / vector_sd for x in vector]

    return vector_std

def standardize(matrix):
    return [standardize1d(vector) for vector in matrix]

In [29]:
simulated_data_list_sd = standardize(simulated_data_list)

In [4]:
%%timeit
simulated_data_list_sd = standardize(simulated_data_list)

7.97 ms ± 28.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**NumPy**

In [7]:
simulated_data.size

100000

In [6]:
simulated_data.shape

(10, 10000)

In [31]:
%%timeit
n_rows = simulated_data.shape[0]

means = simulated_data.mean(axis=1).reshape((n_rows, 1))
stds = simulated_data.std(axis=1).reshape((n_rows, 1))

simulated_data_std = (simulated_data - means) / stds

160 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [20]:
means.shape

(10,)

In [23]:
means.reshape((10, 1))

array([[ 0.04790673],
       [ 5.12854698],
       [-3.98848723],
       [-2.1937482 ],
       [-6.48586082],
       [-9.74727056],
       [ 2.44615233],
       [-1.54783524],
       [ 0.08228267],
       [ 8.99893467]])

In [24]:
simulated_data - means.reshape((10, 1))

array([[-588.86136521,  582.3442177 , -850.28044123, ...,  921.05202814,
          10.42308749, -761.49291579],
       [ -63.74822235,  -89.79287153,  923.07805173, ..., -746.46578641,
        -729.39649998, -265.9508967 ],
       [ 404.79319765, -882.15460697, -946.22642183, ..., -388.76498264,
         716.01905256,  327.24052581],
       ...,
       [ 497.56828704, -647.01112677,  654.59338139, ..., -522.36118802,
         855.28575335,  496.46639058],
       [-322.2363365 , -313.97956971, -279.84498165, ...,  -71.14285719,
          71.76106779,  133.9982277 ],
       [-258.97318248,  226.69325403,  691.401883  , ...,  299.78351125,
         400.31791866, -904.39099707]])

In [18]:
simulated_data - means

ValueError: operands could not be broadcast together with shapes (10,10000) (10,) 

#### Задача 2
Используя средства numpy, проведите вычислительный эксперимент и выясните, можно ли считать статистически значимым результат проверки одной из выдвинутых на прошлом занятии гипотез.

**Data source:** https://www.kaggle.com/datasets/spscientist/students-performance-in-exams.

**Columns:**
- `gender` (binary variable);
- `race/ethnicity` (categorical variable);
- `parental level of education` (categorical variable, student's parents' level of education);
- `lunch` (categorical variable, quality of student's lunch);
- `test preparation course` (categorical variable, whether student managed to complete preparation course for the test);
- `math score` (numeric variable, score for Math test);
- `reading score` (numeric variable, score for Reading part);
- `writing score` (numeric variable, score for Writing part).

#### Задание 1
Загрузите файл StudentsPerformance.csv используя модуль `csv` и возможности модуля `collections`. Не забудьте о том, что для удобства числовые типы данных будет удобнее привести сразу. На выходе должен получиться словарь с ключами-названиями колонок и значениями-колонками.

In [33]:
import csv
from collections import defaultdict

def group(vector, grouping):
    grouped = defaultdict(list)
    
    for v, g in zip(vector, grouping):
        grouped[g].append(v)

    return grouped

def aggregate(grouped, fun):
    aggregated = {key: fun(value) for key, value in grouped.items()}

    return aggregated

path = '../Занятие 1/StudentsPerformance.csv'
int_cols = ['math score', 'reading score', 'writing score']
data = defaultdict(list)

with open(path, mode='r', newline='') as file:
    csvfile = csv.DictReader(file, delimiter=',')

    for dct in csvfile:
        for key, value in dct.items():
            if key in int_cols:
                data[key].append(int(value))
            else:
                data[key].append(value)

In [35]:
len(data['math score'])

1000

In [37]:
math_grouped_by_cource = group(data['math score'], data['test preparation course'])
agg = aggregate(math_grouped_by_cource, np.mean)
agg

{'none': 64.0778816199377, 'completed': 69.69553072625699}

In [38]:
current_diff = agg['completed'] - agg['none']
current_diff

5.617649106319291

In [ ]:
# H0: различия на самом деле нет, все по случайным причинам

# ВОПРОС: какие вообще значения разницы могут получаться по случайным причинам? Насколько текущее значение (реально полученное)
# от них отличается?

# ЗАДАЧА: получить распределение "разниц" при условии, что они возникают только "случайно"

In [39]:
none = np.array(math_grouped_by_cource['none'])
completed = np.array(math_grouped_by_cource['completed'])

In [41]:
concat = np.concatenate((none, completed))
permutated = np.random.permutation(concat)

In [48]:
?np.random.permutation

Docstring:
permutation(x)

Randomly permute a sequence, or return a permuted range.

If `x` is a multi-dimensional array, it is only shuffled along its
first index.

.. note::
    New code should use the
    `~numpy.random.Generator.permutation`
    method of a `~numpy.random.Generator` instance instead;
    please see the :ref:`random-quick-start`.

Parameters
----------
x : int or array_like
    If `x` is an integer, randomly permute ``np.arange(x)``.
    If `x` is an array, make a copy and shuffle the elements
    randomly.

Returns
-------
out : ndarray
    Permuted sequence or array range.

See Also
--------
random.Generator.permutation: which should be used for new code.

Examples
--------
>>> np.random.permutation(10)
array([1, 7, 4, 3, 0, 9, 2, 5, 8, 6]) # random

>>> np.random.permutation([1, 4, 9, 12, 15])
array([15,  1,  9,  4, 12]) # random

>>> arr = np.arange(9).reshape((3, 3))
>>> np.random.permutation(arr)
array([[6, 7, 8], # random
       [0, 1, 2],
       [3, 4, 5]])


In [ ]:
# объединим два массива
# перемешаем значения в них
# разделим обратно на массивы таких же размеров
# посчитаем разницу в средних
# повторить эксперимент N раз

In [ ]:
def simulate_diff(vector1, vector2):
    pass

#### Задача 3
Сделайте данное вам ниже изображение черно-белым.

In [ ]:
from imageio import imread
import matplotlib.pyplot as plt

img1 = imread('./google_maps.png')
plt.imshow(img1);

In [ ]:
# наш код здесь